In [1]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from pyvi import ViTokenizer, ViPosTagger
import string
import os
from tqdm import tqdm
import numpy as np
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/My Drive/Colab Notebooks/Text_Generator

/content/drive/My Drive/Colab Notebooks/Text_Generator


In [5]:
%ls -la

total 180
drwx------ 2 root root  4096 Apr 14 03:53 data/
-rw------- 1 root root 97008 Apr 26 08:14 data_processing.ipynb
drwx------ 2 root root  4096 Apr 14 04:17 model/
-rw------- 1 root root  4482 Apr 26 04:26 scrape_news.ipynb
-rw------- 1 root root 21356 Apr 26 08:29 search_generate_text.ipynb
drwx------ 2 root root  4096 Apr 26 03:59 src/
drwx------ 2 root root  4096 Apr 14 09:19 temp/
-rw------- 1 root root 44172 Apr 26 08:28 text_generator.ipynb


## **Data-processing functions**

In [6]:
def clean_document(doc):
    doc = ViTokenizer.tokenize(doc) #Pyvi Vitokenizer library
    doc = doc.lower() #Lower
    tokens = doc.split() #Split in_to words
    table = str.maketrans('', '', string.punctuation.replace("_", "")) #Remove all punctuation
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word]

    return tokens

print(clean_document('MU vô địch'))

['mu', 'vô_địch']


In [7]:
def preprocess_input(doc):
    tokens = clean_document(doc)
    tokens = tokenizer.texts_to_sequences(tokens)
    tokens = tf.keras.preprocessing.sequence.pad_sequences([tokens], maxlen=50, truncating='pre')

    return np.reshape(tokens, (1,50))

# preprocess_input('MU vô địch')

In [8]:
def top_n_words(text_input, top_n=3):
    tokens = preprocess_input(text_input)
    predictions = model.predict(tokens)[0]

    # Lấy top k dự đoán cao nhất
    top_indices = np.argpartition(predictions, -top_n)[-top_n:]

    top_words = []
    for index in top_indices:
        for word, idx in tokenizer.word_index.items():
            if idx == index:
                top_words.append(word)
                break

    return top_words

In [ ]:
# def create_input(text_data, tokenizer):
#     input_sequences = []
#     for line in text_data:
#         token_list = tokenizer.texts_to_sequences([line])[0]
#         for i in range(1, len(token_list)):
#             n_gram_sequence = token_list[:i+1]
#             input_sequences.append(n_gram_sequence)
#     return input_sequences

## **Load pre-trained model and testing data**

In [9]:
with open('./model/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

with open('./model/sequences_digit.pkl', 'rb') as f:
    sequences_digit = pickle.load(f)

model = tf.keras.models.load_model('./model/51_acc_language_model.h5')

## *Generate text word by word*

In [ ]:
# text_input = "hlv"
# top_3 = top_n_words(text_input, top_n=3)
# print("Top 3 words:")
# print(top_3)

In [ ]:
from ipywidgets import widgets

# Hàm xử lý khi nhấn nút
def button_click(b):
    selected_word = dropbox.value
    input_text.value += " " + selected_word

    # In lại câu input
    print("Input text:", input_text.value)

    # Lấy input text mới
    text_input = input_text.value.strip()

    # Lấy top 3 từ mới
    top_3 = top_n_words(text_input, top_n=3)
    print("Top 3 words:")
    print(top_3)

    # Cập nhật options cho dropbox
    dropbox.options = top_3

# Widget cho input text
input_text = widgets.Text(description="Input text:")

# Widget cho dropbox
top_3 = top_n_words("", top_n=3)  # Khởi tạo dropbox với top 3 từ trống
dropbox = widgets.Dropdown(options=top_3, description="Top 3 words:")

# Widget cho nút
button = widgets.Button(description="Select")

# Gắn hàm xử lý sự kiện khi nhấn nút
button.on_click(button_click)

# Hiển thị các widget
display(input_text, dropbox, button)


## *Google search*

In [15]:
def generate_sentences(text_input, n_words):
    tokens = preprocess_input(text_input)
    generated_sentences = []
    for _ in range(n_words):
        next_digit = np.argmax(model.predict(tokens, verbose=0))
        tokens = np.append(tokens, next_digit)
        tokens = np.delete(tokens, 0)
        tokens = np.reshape(tokens, (1, 50))

    # Mapping to text
    tokens = np.reshape(tokens, (50))
    # print(tokens)
    out_word = []
    for token in tokens:
        for word, index in tokenizer.word_index.items():
            if index == token:
                out_word.append(word)
                break

    return ' '.join(out_word)

def top_n_words(text_input, top_n=3):
    tokens = preprocess_input(text_input)
    predictions = model.predict(tokens)[0]

    # Lấy top k dự đoán cao nhất
    top_indices = np.argpartition(predictions, -top_n)[-top_n:]

    top_words = []
    for index in top_indices:
        for word, idx in tokenizer.word_index.items():
            if idx == index:
                top_words.append(word)
                break

    return top_words

# Khởi tạo danh sách để lưu các câu
generated_sentences = []

# Với mỗi text_input, gọi top_n_words để lấy 3 từ có khả năng.
text_input = "lần đầu cô sinh_viên năm hai"
top_words = top_n_words(text_input, top_n=3)

# Với mỗi từ trong top_words, tạo ra một câu và lưu vào danh sách generated_sentences
for word in top_words:
    new_text_input = text_input + " " + word
    generated_sentence = generate_sentences(new_text_input, n_words=10)
    generated_sentences.append(generated_sentence)

# In ra các câu đã được tạo ra
for i, sentence in enumerate(generated_sentences, start=1):
    print("Generated sentence", i, ":", sentence)

1/1 [==============================] - 0s 95ms/step
Generated sentence 1 : lần đầu cô sinh_viên năm hai lần hai một trường đh ở hà_nội bạn trai đánh lần
Generated sentence 2 : lần đầu cô sinh_viên năm hai năm điều_trị trong một gia_đình anh mắc lây ở giúp đi
Generated sentence 3 : lần đầu cô sinh_viên năm hai một trường đh ở hà_nội bạn trai đánh lần bực_bội người_yêu


In [ ]:
from ipywidgets import interact, widgets, Output

# Hàm xử lý khi textbox thay đổi giá trị
def on_text_change(change):
    text_output.clear_output()  # Xóa nội dung hiện tại của widget Output
    generate_sentences(change, 5)
    with text_output:  # Sử dụng with để hiển thị nội dung mới
        print(change.new)

# Tạo một textbox
text_input = widgets.Text(description="Input text:")

# Tạo một output widget để hiển thị giá trị của textbox
text_output = Output()

# Gắn hàm xử lý sự kiện khi textbox thay đổi giá trị
text_input.observe(on_text_change, names='value')

# Hiển thị textbox và output widget
display(text_input, text_output)


In [ ]:
from ipywidgets import interact, widgets, Output

# Hàm xử lý khi textbox thay đổi giá trị
def on_text_change(change):
    text_input = change.new.strip()  # Lấy giá trị mới từ textbox và loại bỏ khoảng trắng ở đầu và cuối
    if not text_input:
        return  # Nếu text_input là chuỗi rỗng, không làm gì cả

    # Kiểm tra xem có phím space trong text_input không
    if ' ' in text_input:
        # Lấy các từ từ text_input
        words = text_input.split()

        # Lấy từ cuối cùng trong chuỗi (từ vừa nhập)
        last_word = words[-1]

        # Kiểm tra xem từ vừa nhập có kích thước lớn hơn 1 không
        if len(last_word) > 1:
            # Nếu có, thực hiện generate
            top_words = top_n_words(text_input, top_n=3)

            # Xóa nội dung hiện tại trong output widget
            with output:
                output.clear_output(wait=False)

            # Tạo lại các câu mới
            generated_sentences.clear()
            for word in top_words:
                new_text_input = text_input + " " + word
                generated_sentence = generate_sentences(new_text_input, n_words=10)
                generated_sentences.append(generated_sentence)

            # In ra các câu mới
            with output:
                for i, sentence in enumerate(generated_sentences, start=1):
                    print("Generated sentence", i, ":", sentence)

# Tạo một textbox
text_input = widgets.Text(description="Input text:")

# Widget Output để hiển thị các câu được tạo ra
output = Output()

# Gắn hàm xử lý sự kiện khi textbox thay đổi giá trị
text_input.observe(on_text_change, names='value')

# Hiển thị textbox và output widget
display(text_input, output)


Text(value='', description='Input text:')

Output()

## **Evaluate the result**

In [ ]:
# test_loss, test_acc = model.evaluate(X, y, verbose=1)

In [ ]:
# !pip install sequence-evaluate
# !pip install rouge
# !pip install sentence-transformers

In [ ]:
# from seq_eval import SeqEval
# evaluator = SeqEval()

In [ ]:
candidates = ['chỉ trong vòng 20 phút đầu gđtla bị hagl dẫn trước đến 2 0 trong lúc chủ nhà tấn_công bế_tắc với cặp tiền_đạo anh_em nhà rodrigues']

references = ['chỉ trong vòng 20 phút đầu, gđtla bị hagl dẫn trước đến 2 0 trong lúc chủ nhà tấn công bế tắc với cặp tiền đạo anh em nhà rodrigues']

In [ ]:
# scores = evaluator.evaluate(candidates, references, verbose=True)